In [109]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [110]:
def calculate_null_percentage(df):

    return df.isnull().sum(axis=1) / len(df.columns)

In [111]:

def remove_high_null_rows(df, threshold=0.8):

    null_percentage_per_row = calculate_null_percentage(df)
    df_cleaned = df[null_percentage_per_row < threshold].copy()
    
    print(f"Original data shape: {df.shape}")
    print(f"Cleaned data shape: {df_cleaned.shape}")
    print(f"Removed {len(df) - len(df_cleaned)} rows with >{threshold*100}% null values")
    
    return df_cleaned

In [112]:

def remove_null_essential_columns(df, essential_columns=['title']):

    df_filtered = df.copy()
    for col in essential_columns:
        if col in df_filtered.columns:
            df_filtered = df_filtered[df_filtered[col].notna()]
    
    return df_filtered

In [116]:


def get_sector_keywords():
    """
    Define sector keywords dictionary
    
    Returns:
        dict: Sector keywords mapping
    """
    return {
        'Informatique': [
            'informatique', 'it', 'tech', 'digital', 'software', 'développeur', 
            'developer', 'programmeur', 'java', 'python', 'web', 'application', 
            'système', 'network', 'cyber', 'sécurité', 'data', 'base de données', 
            'sql', 'oracle', 'mongodb', 'mysql', 'php', 'javascript', 'html', 
            'css', 'react', 'angular', 'node'
        ],
        'Telecom': [
            'telecom', 'télécommunication', 'réseau', 'network', 'wifi', '4g', 
            '5g', 'radio', 'antenne', 'infrastructure', 'telecommunications'
        ],
        'Finance': [
            'finance', 'financier', 'comptabilité', 'banking', 'banque', 
            'assurance', 'investment', 'trading', 'audit', 'comptable'
        ],
        'Marketing': [
            'marketing', 'digital marketing', 'communication', 'publicité', 
            'social media', 'seo', 'sem', 'brand', 'emailing'
        ],
        'Conseil': [
            'consultant', 'consulting', 'conseil', 'advisory', 'strategy', 
            'stratégie', 'expertise'
        ],
        'Production': [
            'production', 'manufacturing', 'fabrication', 'usine', 'industrie', 
            'qualité', 'maintenance', 'industrial'
        ],
        'Ressources Humaines': [
            'rh', 'ressources humaines', 'recrutement', 'hr', 'human resources', 
            'talent', 'recruitment'
        ],
        'Vente': [
            'vente', 'sales', 'commercial', 'business development', 
            'account manager', 'commerce'
        ],
        'Support': [
            'support', 'helpdesk', 'assistance', 'service client', 
            'customer service', 'technical support'
        ],
        'Gestion': [
            'gestion', 'management', 'administration', 'operational', 
            'project manager', 'chef de projet', 'manager'
        ]
    }


In [117]:
def extract_sector_from_text(text, sector_keywords):

    if not text or pd.isna(text):
        return "Autre"
    
    text_lower = text.lower()
    matched_sectors = []
    
    for sector, keywords in sector_keywords.items():
        for keyword in keywords:
            if keyword in text_lower:
                matched_sectors.append(sector)
                break
    
    if matched_sectors:
        return Counter(matched_sectors).most_common(1)[0][0]
    
    return "Autre"

In [118]:


def extract_sector(title, description, existing_sector):

    # First check if existing sector is available and valid
    if pd.notna(existing_sector) and existing_sector.strip():
        return existing_sector.strip()
    
    # Combine title and description for analysis
    text = ""
    if pd.notna(title):
        text += str(title) + " "
    if pd.notna(description):
        text += str(description) + " "
    
    sector_keywords = get_sector_keywords()
    return extract_sector_from_text(text, sector_keywords)

In [119]:

def get_job_category_keywords():
    """
    Define job category keywords dictionary
    
    Returns:
        dict: Job category keywords mapping
    """
    return {
        'Science & IA': [
            'data scientist', 'machine learning', 'ai', 'artificial intelligence', 
            'deep learning', 'ml', 'data analysis', 'analytics', 'big data', 
            'data mining', 'neural network', 'algorithm', 'research', 'scientist',
            'intelligence artificielle', 'apprentissage automatique'
        ],
        'Développement': [
            'développeur', 'developer', 'programmeur', 'coding', 'programming', 
            'software engineer', 'full stack', 'backend', 'frontend', 
            'web developer', 'mobile developer', 'fullstack'
        ],
        'Gestion de Projet': [
            'chef de projet', 'project manager', 'scrum master', 'product owner', 
            'agile', 'kanban', 'release manager'
        ],
        'Infrastructure': [
            'devops', 'system admin', 'network', 'infrastructure', 'cloud', 
            'server', 'virtualization', 'security', 'réseau', 'système',
            'ingénieur système', 'administrateur système'
        ],
        'Analyse': [
            'analyst', 'analyste', 'business analyst', 'data analyst', 
            'financial analyst', 'business intelligence', 'bi'
        ],
        'Consultation': [
            'consultant', 'expert', 'specialist', 'advisor', 'conseil',
            'expertise', 'consulting'
        ],
        'Support': [
            'support', 'helpdesk', 'technical support', 'maintenance', 
            'assistance', 'help desk'
        ],
        'Management': [
            'manager', 'responsable', 'director', 'lead', 'supervisor', 
            'chef', 'directeur', 'coordinateur'
        ]
    }

In [120]:

def extract_job_category_from_text(text, job_category_keywords):

    if not text or pd.isna(text):
        return "Autre"
    
    text_lower = text.lower()
    matched_categories = []
    
    for category, keywords in job_category_keywords.items():
        for keyword in keywords:
            if keyword in text_lower:
                matched_categories.append(category)
                break
    
    if matched_categories:
        return Counter(matched_categories).most_common(1)[0][0]
    
    return "Autre"


In [121]:

def extract_job_category(title, description):

    # Combine title and description for analysis
    text = ""
    if pd.notna(title):
        text += str(title) + " "
    if pd.notna(description):
        text += str(description) + " "
    
    job_category_keywords = get_job_category_keywords()
    return extract_job_category_from_text(text, job_category_keywords)

In [122]:

def get_stop_words():
    """
    Get list of stop words to filter out from competencies
    
    Returns:
        set: Set of stop words
    """
    return {
        'ET', 'OU', 'DE', 'LA', 'LE', 'DU', 'DES', 'UN', 'UNE', 'AVEC', 
        'POUR', 'DANS', 'SUR', 'AND', 'OR', 'THE', 'A', 'AN', 'WITH', 
        'FOR', 'IN', 'ON', 'OF'
    }

In [123]:

def split_competencies(competence_text):

    if pd.isna(competence_text) or not str(competence_text).strip():
        return []
    
    # Split by common separators
    competencies = re.split(r'[,\-\s]+', str(competence_text).upper())
    return competencies

In [124]:


def filter_competencies(competencies):

    stop_words = get_stop_words()
    
    filtered = []
    for comp in competencies:
        comp_clean = comp.strip()
        if (comp_clean and 
            comp_clean not in stop_words and 
            len(comp_clean) > 1 and
            not comp_clean.isdigit()):
            filtered.append(comp_clean)
    
    return filtered

In [125]:


def clean_competence(competence_text):

    competencies = split_competencies(competence_text)
    filtered_competencies = filter_competencies(competencies)
    
    return ', '.join(filtered_competencies) if filtered_competencies else ""


In [126]:

def apply_all_extractions(df):

    df_processed = df.copy()
    
    print("Applying sector extraction...")
    df_processed['sector_extracted'] = df_processed.apply(
        lambda row: extract_sector(row.get('title'), row.get('description'), row.get('sector')), 
        axis=1
    )
    
    print("Applying job category extraction...")
    df_processed['job_category'] = df_processed.apply(
        lambda row: extract_job_category(row.get('title'), row.get('description')), 
        axis=1
    )
    
    print("Applying competence cleaning...")
    if 'Competence' in df_processed.columns:
        df_processed['competence_cleaned'] = df_processed['Competence'].apply(clean_competence)
    
    return df_processed


In [127]:

def select_final_columns(df):
    
    final_columns = [
        'title', 'description', 'sector_extracted', 'job_category', 
        'competence_cleaned', 'company_name', 'Region', 'contract_type', 
        'niveau d\'études requis', 'niveau d\'expérience', 'posted_date', 'remote_work'
    ]
    
    # Only keep columns that exist in the dataframe
    available_columns = [col for col in final_columns if col in df.columns]
    return df[available_columns].copy()

In [128]:


def print_statistics(df):

    print(f"\nFinal processed data shape: {df.shape}")
    
    if 'sector_extracted' in df.columns:
        print("\n=== SECTOR DISTRIBUTION ===")
        print(df['sector_extracted'].value_counts())
    
    if 'job_category' in df.columns:
        print("\n=== JOB CATEGORY DISTRIBUTION ===")
        print(df['job_category'].value_counts())
    
    if 'competence_cleaned' in df.columns:
        print("\n=== SAMPLE OF CLEANED COMPETENCIES ===")
        sample_competencies = df[df['competence_cleaned'] != '']['competence_cleaned'].head(10)
        for i, comp in enumerate(sample_competencies, 1):
            print(f"{i}. {comp}")

In [129]:
def process_csv_file(csv_file_path, output_file_path='cleaned_job_data.csv'):

    try:
        # Step 1: Read CSV
        print("Reading CSV file...")
        df = pd.read_csv(csv_file_path)
        print(f"Loaded {len(df)} rows with {len(df.columns)} columns")
        
        # Step 2: Remove high null rows
        print("\nCleaning data...")
        df_cleaned = remove_high_null_rows(df, threshold=0.8)
        
        # Step 3: Apply all extractions
        print("\nExtracting information...")
        df_processed = apply_all_extractions(df_cleaned)
        
        # Step 4: Remove rows with null essential columns
        print("\nRemoving rows with null essential data...")
        df_final = remove_null_essential_columns(df_processed, ['title'])
        
        # Step 5: Select final columns
        print("\nSelecting final columns...")
        df_output = select_final_columns(df_final)
        
        # Step 6: Save results
        print(f"\nSaving results to {output_file_path}...")
        df_output.to_csv(output_file_path, index=False, encoding='utf-8')
        
        # Step 7: Print statistics
        print_statistics(df_output)
        
        print(f"\n✅ Processing completed successfully!")
        print(f"📁 Output saved to: {output_file_path}")
        
        return df_output
        
    except FileNotFoundError:
        print(f"❌ Error: File '{csv_file_path}' not found.")
        return None
    except Exception as e:
        print(f"❌ Error processing file: {str(e)}")
        return None


In [133]:
cleaned_data = process_csv_file('merged_jobs.csv', 'cleaned_merged_jobs.csv')
    
if cleaned_data is not None:
        print("\n=== SAMPLE OF PROCESSED DATA ===")
        sample_columns = ['title', 'sector_extracted', 'job_category', 'competence_cleaned']
        available_sample_columns = [col for col in sample_columns if col in cleaned_data.columns]
       

Reading CSV file...
Loaded 3003 rows with 17 columns

Cleaning data...
Original data shape: (3003, 17)
Cleaned data shape: (3003, 17)
Removed 0 rows with >80.0% null values

Extracting information...
Applying sector extraction...
Applying job category extraction...
Applying competence cleaning...

Removing rows with null essential data...

Selecting final columns...

Saving results to cleaned_merged_jobs.csv...

Final processed data shape: (2958, 12)

=== SECTOR DISTRIBUTION ===
sector_extracted
Informatique                                    1631
Autre                                            422
['Informatique']                                 313
Gestion                                          130
Conseil                                          125
Support                                           62
['Informatique', 'Internet / Multimédia']         57
Production                                        43
Telecom                                           37
Ressources Humaines   

In [134]:
cleaned_data[available_sample_columns].head()

,title,sector_extracted,job_category,competence_cleaned
0,Ingénieur Full-Stack | Casablanca (Maroc),['Informatique'],Autre,
1,Test Manager Anglophone (H/F) | Rabat (Maroc),['Informatique'],Management,
2,Digital Campaign Production Manager | Casablan...,"['Informatique', 'Telecom']",Science & IA,
3,Click & Webmaster | Casablanca (Maroc),"['Informatique', 'Telecom']",Autre,
4,Click User Support Manager | Casablanca (Maroc),"['Informatique', 'Telecom']",Support,
